In [1]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image 
from IPython.core.display import Image, display
from collections import Counter
import pandas as pd
import pymongo
import bson
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import re
import random
from itertools import permutations
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags


import pickle
from datetime import datetime
from collections import Counter

# SEARCH GRAPHS

In [3]:
collector_cik_trash = []
for i in db_client["tweets_pipeline_v2"].find({
                                               "status":"graphicone_search",
                                                       'source': 'sec',
                                                "cik": {"$exists": True}
                                                   }):
    collector_cik_trash.append(i['cik'])
print(len(Counter(collector_cik_trash)))
Counter(collector_cik_trash).most_common()    

1408


[('19617', 67),
 ('109380', 45),
 ('886982', 39),
 ('920112', 33),
 ('1043509', 33),
 ('1521951', 31),
 ('356171', 27),
 ('1224608', 26),
 ('1376139', 25),
 ('40211', 25),
 ('1477449', 25),
 ('1127703', 24),
 ('1159508', 22),
 ('7789', 21),
 ('101382', 21),
 ('767405', 20),
 ('1455684', 19),
 ('23217', 19),
 ('852772', 19),
 ('721994', 19),
 ('840489', 18),
 ('1606757', 18),
 ('54480', 17),
 ('230557', 17),
 ('785161', 17),
 ('831259', 17),
 ('1084961', 16),
 ('1102112', 16),
 ('1611647', 16),
 ('1025835', 16),
 ('76605', 16),
 ('719739', 16),
 ('1341317', 15),
 ('1420811', 15),
 ('1393066', 15),
 ('715072', 15),
 ('1367859', 15),
 ('1528129', 15),
 ('1604481', 15),
 ('818033', 15),
 ('1481582', 15),
 ('737468', 15),
 ('354647', 14),
 ('1458412', 14),
 ('1289490', 14),
 ('34782', 14),
 ('1488813', 14),
 ('946647', 14),
 ('714310', 14),
 ('924901', 14),
 ('920522', 13),
 ('1300050', 13),
 ('750686', 13),
 ('1475348', 13),
 ('1380846', 13),
 ('764622', 13),
 ('1413837', 13),
 ('1519401',

In [9]:
df = pd.DataFrame.from_dict(Counter(collector_cik_trash).most_common())
df.columns = ['cik', 'count']
df

,cik,count
0,19617,67
1,109380,45
2,886982,39
3,920112,33
4,1043509,33
...,...,...
1403,1474098,1
1404,1644903,1
1405,1624794,1
1406,1561894,1


In [28]:
sec = pd.read_csv("data-1596200464136.csv")
# sec.set_index("cik", inplace = True)
sec.cik = sec.cik.astype(str)
sec

,ticker,name,cik,legal_name,short_description,long_description
0,EPR,EPR Properties,1045450,EPR PROPERTIES,EPR Properties is a real estate investment tru...,EPR Properties is a real estate investment tru...
1,SAVE,Spirit Airlines Inc,1498710,"Spirit Airlines, Inc.","Spirit Airlines, Inc. is an ultra low-cost, lo...","Spirit Airlines, Inc. provides low-fare airlin..."
2,CPTA,Capitala Finance Corp,1571329,Capitala Finance Corp.,NaN,NaN
3,OII,"Oceaneering International, Inc.",73756,OCEANEERING INTERNATIONAL INC,"Oceaneering International, Inc. provides engin...","Oceaneering International, Inc. provides engin..."
4,CVI,CVR Energy Inc,1376139,CVR ENERGY INC,"CVR Energy, Inc. engages in the provision of e...","CVR Energy, Inc., through its subsidiaries, en..."
...,...,...,...,...,...,...
5847,ADNT,Adient plc,1670541,Adient plc,Adient plc is an automotive seating supplier t...,"Adient plc designs, manufactures, and markets ..."
5848,INSW,International Seaways Inc,1679049,"International Seaways, Inc.","International Seaways, Inc. operates a fleet o...","International Seaways, Inc. owns and operates ..."
5849,WSRC,Western Sierra Resource Corp,42050,WESTERN SIERRA MINING CORP,Western Sierra Resource Corporation engages in...,Western Sierra Resource Corporation engages in...
5850,ATRX,Adhera Therapeutics Inc,737207,"Marina Biotech, Inc.","Adhera Therapeutics, Inc., a biopharmaceutical...","Adhera Therapeutics, Inc., a biopharmaceutical..."


In [43]:
def add_ticker(x, column):
    ticker = sec[sec.cik == x][column].values
    if ticker:
        return ticker[0]

In [44]:
df["ticker"] = df.cik.apply(lambda x: add_ticker(x, 'ticker'))
df["name"] = df.cik.apply(lambda x: add_ticker(x, 'name'))

In [45]:
df

,cik,count,ticker,name
0,19617,67,JPM,JPMorgan Chase & Co.
1,109380,45,ZION,Zions Bancorporation N.A
2,886982,39,GS,"Goldman Sachs Group, Inc."
3,920112,33,HTLF,"Heartland Financial USA, Inc."
4,1043509,33,SAH,"Sonic Automotive, Inc."
...,...,...,...,...
1403,1474098,1,PEB,Pebblebrook Hotel Trust
1404,1644903,1,YCBD,cbdMD Inc
1405,1624794,1,CSWI,CSW Industrials Inc
1406,1561894,1,HASI,Hannon Armstrong Sustainable Infrastructure ca...


In [46]:
df.to_excel("vas_sec_comps.xlsx")

In [2]:
db_client["tweets_pipeline_v2"].find({
                                               "status":"trasher",
                                                       'source': 'sec',
                                                   }).count()

7511

In [79]:
db_client["tweets_pipeline_v2"].find({
                                               "status":"bd_search",
                                                       'source': 'sec',
                                                   }).count()

24

In [71]:
collector_cik_trash = []
for i in db_client["tweets_pipeline_v2"].find({
                                               "status":"trasher",
                                                       'source': 'sec',
                                                   }):
    collector_cik_trash.append(i['cik'])
print(len(Counter(collector_cik_trash)))
Counter(collector_cik_trash).most_common()    

2036


[('831001', 213),
 ('895421', 86),
 ('1203464', 72),
 ('886982', 65),
 ('1000275', 55),
 ('1001290', 46),
 ('19617', 45),
 ('1628171', 43),
 ('1631574', 41),
 ('918608', 39),
 ('72971', 33),
 ('1681087', 29),
 ('947263', 27),
 ('800166', 27),
 ('1364128', 26),
 ('1173643', 26),
 ('1163302', 25),
 ('1744659', 24),
 ('1231346', 24),
 ('1300050', 23),
 ('1025835', 22),
 ('1713930', 20),
 ('1080709', 19),
 ('1705259', 19),
 ('70858', 18),
 ('1656634', 18),
 ('1720725', 18),
 ('1610618', 18),
 ('785161', 18),
 ('1326110', 18),
 ('9631', 18),
 ('1508844', 18),
 ('1654151', 17),
 ('1682852', 17),
 ('749098', 17),
 ('1720580', 17),
 ('1697532', 16),
 ('1468748', 16),
 ('874766', 16),
 ('1718108', 16),
 ('73088', 16),
 ('701818', 16),
 ('1279704', 16),
 ('898174', 16),
 ('1579428', 15),
 ('1597264', 15),
 ('1714798', 15),
 ('1323404', 15),
 ('20520', 15),
 ('1535955', 15),
 ('1477449', 15),
 ('920112', 14),
 ('1709401', 14),
 ('1382101', 14),
 ('1548187', 14),
 ('1526119', 14),
 ('1235010', 14)

In [84]:
collector_cik_clean = []
for i in db_client["tweets_pipeline_v2"].find({
                                               "status":"bd_search",
                                                       'source': 'sec',
                                                   }):
    collector_cik_clean.append(i['cik'])
print(len(Counter(collector_cik_clean)))    
Counter(collector_cik_clean).most_common()    

7


[('1706350', 7),
 ('1640967', 4),
 ('1812768', 2),
 ('1741687', 1),
 ('1654124', 1),
 ('1794790', 1),
 ('1786108', 1)]

In [66]:
dict(Counter(collector_cik_clean))["320193"]

2

In [68]:
db = db_client["tweets_pipeline_v2"].find({
                                           "status":"bd_search",
                                                   'source': 'sec',
    "cik": "19617"
                                               })
for i in db:
    print(i)

{'_id': ObjectId('5f10547f0c043804ffc262c9'), 'name': 'JPMORGAN CHASE & CO', 'cik': '19617', 'exec_date': datetime.datetime(2020, 7, 16, 12, 12, 51, 17000), 'img_urls': 'https://www.sec.gov/Archives/edgar/data/19617/000001961720000257/chart-2a52f1787afa537fa3a.jpg', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 2, 25, 0, 0), 'source': 'sec', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'bd_search', 'extracted_text': 'JPMorgan Chase KBW Bank S&P Financials S&P 500 250 225 200 175 150 125 ... .. 100 75 2014 2015 2016 2017 2018 2019', 'bert_tags': ['banking', 'large caps'], 'industries_from_bert': ['Banks insurance and finance'], 'shifts_from_bert': [], 'trasher': [], 'pipelined_ts': datetime.datetime(2020, 7, 16, 13, 22, 7, 150000), 'aws_answer': [{'Score': 0.9993875622749329, 'Type': 'ORGANIZATION', 'Text': 'JPMorgan Chase', 'BeginOffset': 0, 'EndOffset': 14}, {'Score': 0.9995691776275635, 'Type': 'ORGANIZATION', 'Text': 'KBW Bank', 'BeginOffset': 15, 'EndOffset': 23}, {'Sc

In [ ]:
for i in 


In [59]:
print(len(set(collector_cik_clean)-set(collector_cik_trash)))
print(len(set(collector_cik_trash)-set(collector_cik_clean)))

438
815


In [27]:
def downloadSec():
    db = db_client["tweets_pipeline_v2"].find({
                                                       'source': 'dailyshot',
                                                   }).sort([("TIME_FIELD_FOR_SORTING", pymongo.DESCENDING)])

    images_links = []
    for i in db:
        print(i["pipelined_ts"], i["TIME_FIELD_FOR_SORTING"])
        images_links.append([str(i["_id"]), i["img_urls"]])
    return images_links

In [28]:
collector = downloadSec()[:100]

2020-08-25 12:45:22.769000 2020-08-03 00:00:00
2020-08-25 12:45:25.233000 2020-08-03 00:00:00
2020-08-25 12:45:29.020000 2020-08-03 00:00:00
2020-08-25 12:45:30.516000 2020-08-03 00:00:00
2020-08-25 12:45:31.317000 2020-08-03 00:00:00
2020-08-25 12:45:33.469000 2020-08-03 00:00:00
2020-08-25 12:45:48.887000 2020-08-03 00:00:00
2020-08-25 12:45:53.312000 2020-08-03 00:00:00
2020-08-25 13:19:59.909000 2020-08-03 00:00:00
2020-08-25 13:20:05.677000 2020-08-03 00:00:00
2020-08-25 13:20:06.811000 2020-08-03 00:00:00
2020-08-25 13:20:07.186000 2020-08-03 00:00:00
2020-08-25 13:20:10.632000 2020-08-03 00:00:00
2020-08-25 13:20:17.230000 2020-08-03 00:00:00
2020-08-25 13:20:26.296000 2020-08-03 00:00:00
2020-08-25 13:20:26.933000 2020-08-03 00:00:00
2020-07-10 13:41:23.718000 2020-07-02 00:00:00
2020-07-10 13:41:26.424000 2020-07-02 00:00:00
2020-07-10 13:41:26.542000 2020-07-02 00:00:00
2020-07-10 13:41:29.412000 2020-07-02 00:00:00
2020-07-10 13:41:38.755000 2020-07-02 00:00:00
2020-07-10 13

2020-08-25 22:50:58.217000 2020-02-20 00:00:00
2020-08-25 22:51:00.660000 2020-02-20 00:00:00
2020-08-26 23:56:06.023000 2020-02-19 00:00:00
2020-08-26 23:56:11.211000 2020-02-19 00:00:00
2020-08-26 23:56:13.624000 2020-02-19 00:00:00
2020-08-26 23:56:15.872000 2020-02-19 00:00:00
2020-08-26 23:56:17.775000 2020-02-19 00:00:00
2020-08-26 23:56:20.915000 2020-02-19 00:00:00
2020-08-25 23:05:16.478000 2020-02-18 00:00:00
2020-08-25 23:05:19.757000 2020-02-18 00:00:00
2020-08-25 23:05:33.298000 2020-02-18 00:00:00
2020-08-25 23:05:34.303000 2020-02-18 00:00:00
2020-08-25 23:05:42.028000 2020-02-18 00:00:00
2020-08-25 23:05:45.200000 2020-02-18 00:00:00
2020-08-25 23:08:37.797000 2020-02-18 00:00:00
2020-08-25 23:08:53.380000 2020-02-18 00:00:00
2020-08-27 05:55:58.440000 2020-02-17 00:00:00
2020-08-27 05:56:26.559000 2020-02-17 00:00:00
2020-08-27 05:56:31.128000 2020-02-17 00:00:00
2020-08-27 05:56:34.843000 2020-02-17 00:00:00
2020-08-27 05:59:12.965000 2020-02-17 00:00:00
2020-08-25 14

In [29]:
totalValidated = pd.DataFrame(collector, columns=["ID", "url"])    
totalValidated

,ID,url
0,5f4507e22eb24ab18d8caacb,https://thedailyshot.com/wp-content/uploads/CH...
1,5f4507e5eb204418fc74b566,https://thedailyshot.com/wp-content/uploads/EN...
2,5f4507e956021779e8b06cc6,https://thedailyshot.com/wp-content/uploads/US...
3,5f4507ea03068e29a301d174,https://thedailyshot.com/wp-content/uploads/CH...
4,5f4507eb2e5b3dcc06db566d,https://thedailyshot.com/wp-content/uploads/CH...
...,...,...
95,5f45ac0e56897212a033a885,https://si.wsj.net/public/resources/images/B3-...
96,5f47639c171a5af81314f495,https://si.wsj.net/public/resources/images/B3-...
97,5f4763a8adf2c9614ca65250,https://si.wsj.net/public/resources/images/B3-...
98,5f4763e69d398547c7d1efaf,https://si.wsj.net/public/resources/images/B3-...


In [30]:
from tqdm import tqdm
import os
tqdm.pandas()
import requests


In [31]:
def download(folderName,_id, extension,r, MainDir):
    if extension in [".jpg", ".png"]:
        if not os.path.exists(MainDir+"/"+folderName):
            os.makedirs(MainDir+"/"+folderName)
        path = MainDir+folderName+"/"+str(_id) + extension
        print(path)
        with open(path, 'wb') as f:
            f.write(r.content)

In [32]:
def FolderPreparator(x, MainDir):
    
    if not os.path.exists(MainDir):
        os.mkdir(MainDir)    
        
    folders = x[x==True].index.values
    _id, url = x["ID"], x["url"]
    extension = url[-4:]
    r = requests.get(url)
    reason_folder = ""
    download(reason_folder,_id, extension, r, MainDir)

In [33]:
mainDir = "daily-shot"

In [34]:
totalValidated = totalValidated.progress_apply(lambda x: FolderPreparator(x, mainDir), axis=1)

  4%|▍         | 4/100 [00:00<00:06, 15.63it/s]

daily-shot/5f4507e22eb24ab18d8caacb.png
daily-shot/5f4507e5eb204418fc74b566.png
daily-shot/5f4507e956021779e8b06cc6.png


  6%|▌         | 6/100 [00:00<00:06, 15.22it/s]

daily-shot/5f4507ea03068e29a301d174.png
daily-shot/5f4507eb2e5b3dcc06db566d.png
daily-shot/5f4507ed0e0e02011a02b9a7.png


 10%|█         | 10/100 [00:00<00:06, 13.49it/s]

...

daily-shot/5f45ac0e56897212a033a885.png
daily-shot/5f47639c171a5af81314f495.png


100%|██████████| 100/100 [00:18<00:00,  5.57it/s]

daily-shot/5f4763a8adf2c9614ca65250.png
daily-shot/5f4763e69d398547c7d1efaf.png


100%|██████████| 100/100 [00:18<00:00,  5.29it/s]

daily-shot/5f4763fdab69d08cb0276b6d.png


In [55]:
!zip -r sec_vas.zip sec_vas

  adding: sec_vas/ (stored 0%)
  adding: sec_vas/5f1149e1e11e38fd7e3ea03a.jpg (deflated 16%)
  adding: sec_vas/5f10d564623ea22e665052f7.jpg (deflated 31%)
  adding: sec_vas/5f11f25f5a47d0bdc8b0ae06.jpg (deflated 12%)
  adding: sec_vas/5f11315267b7678bde96e01f.jpg (deflated 22%)
  adding: sec_vas/5f11e8942a4d1439f598d086.jpg (deflated 15%)
  adding: sec_vas/5f10c9f56be62f0c47bb9c0d.jpg (deflated 24%)
  adding: sec_vas/5f10c142737b95768fc2f780.jpg (deflated 15%)
  adding: sec_vas/5f10cbb39ed11f7e2b7ec818.jpg (deflated 9%)
  adding: sec_vas/5f10cf64bca8089faba0ba45.jpg (deflated 16%)
  adding: sec_vas/5f11cf662c256c60a6d8b512.jpg (deflated 26%)
  adding: sec_vas/5f11ba317cc2424ba1a875c8.jpg (deflated 14%)
  adding: sec_vas/5f11273e7abe47ab35fc8cdf.jpg (deflated 35%)
  adding: sec_vas/5f10463167ba782a6e4436fc.jpg (deflated 12%)
  adding: sec_vas/5f11f1e0ef1b6a226b568976.jpg (deflated 7%)
  adding: sec_vas/5f11e4c22a4513461b57c505.jpg (deflated 22%)
  adding: sec_vas/5f10a01960e2ed404ffe834

  adding: sec_vas/5f112baf57094240754c564b.jpg (deflated 10%)
  adding: sec_vas/5f112bdabd44bdc9c956e8af.jpg (deflated 38%)
  adding: sec_vas/5f11375cdd6a7867480247e0.jpg (deflated 21%)
  adding: sec_vas/5f10cd4ba7307e7cc65b41d4.jpg (deflated 30%)
  adding: sec_vas/5f11c497641b74a67e886145.jpg (deflated 27%)
  adding: sec_vas/5f10a16841393c65d2a695e1.jpg (deflated 35%)
  adding: sec_vas/5f104b315f6293c0cb7cccc1.jpg (deflated 31%)
  adding: sec_vas/5f11ea122b621e886e382009.jpg (deflated 49%)
  adding: sec_vas/5f1110e551a6f61f1eba4f11.jpg (deflated 33%)
  adding: sec_vas/5f11ce73d34a571cd89d9c7b.jpg (deflated 13%)
  adding: sec_vas/5f10b3a628c86d1525fca0ac.jpg (deflated 27%)
  adding: sec_vas/5f1130e8090d70c5bf8c9588.jpg (deflated 8%)
  adding: sec_vas/5f119d47610e22397254cf7d.jpg (deflated 44%)
  adding: sec_vas/5f10d52364369eb40e9efda6.jpg (deflated 6%)
  adding: sec_vas/5f11c2a8d59b67afd09322de.jpg (deflated 17%)
  adding: sec_vas/5f122da24cbe8d517c2c262d.jpg (deflated 10%)
  adding: 

  adding: sec_vas/5f11e4ad8d4243ae181470d0.jpg (deflated 5%)
  adding: sec_vas/5f121fadf66a7fc6ab0f1458.jpg (deflated 21%)
  adding: sec_vas/5f11ea0d302639a00f08c263.jpg (deflated 4%)
  adding: sec_vas/5f113443dc7a5f351a0965d1.jpg (deflated 31%)
  adding: sec_vas/5f1223b8308a45875ca49c12.jpg (deflated 4%)
  adding: sec_vas/5f11c1dfe15bf738604cd9c6.jpg (deflated 12%)
  adding: sec_vas/5f105576606631aa548c0162.jpg (deflated 14%)
  adding: sec_vas/5f11d03bd26206adce319cda.jpg (deflated 9%)
  adding: sec_vas/5f11057811acafc679bf1a50.jpg (deflated 10%)
  adding: sec_vas/5f114d6d2ab97a1ad83d31b9.jpg (deflated 24%)
  adding: sec_vas/5f11c47a5fd44eb85fe508ba.jpg (deflated 17%)
  adding: sec_vas/5f10dc00c7b71fac48750462.jpg (deflated 14%)
  adding: sec_vas/5f11e17a80a3f757522738bf.jpg (deflated 21%)
  adding: sec_vas/5f10bbc5c8026da04a29db03.jpg (deflated 43%)
  adding: sec_vas/5f117bcf64e300a8a2b100df.jpg (deflated 19%)
  adding: sec_vas/5f11c678641b74a67e886149.jpg (deflated 17%)
  adding: se

  adding: sec_vas/5f121035078af63862ef89ac.jpg (deflated 20%)
  adding: sec_vas/5f104727936e1755f425d616.jpg (deflated 14%)
  adding: sec_vas/5f11d180fb3f37b3630121cc.jpg (deflated 15%)
  adding: sec_vas/5f10d10689ade70c6d9e2d95.jpg (deflated 19%)
  adding: sec_vas/5f11c0bb91cdba5cb96dcbfb.jpg (deflated 15%)
  adding: sec_vas/5f10fa0b7162a368025c2532.jpg (deflated 5%)
  adding: sec_vas/5f10cc48cb51aec63556d2dc.jpg (deflated 10%)
  adding: sec_vas/5f110fe8975cd810d8087298.jpg (deflated 19%)
  adding: sec_vas/5f10caf37bde98ca37c6a367.jpg (deflated 10%)
  adding: sec_vas/5f106347c310f6db3ba8f1ea.jpg (deflated 36%)
  adding: sec_vas/5f118a77c2c70a599bec697d.jpg (deflated 22%)
  adding: sec_vas/5f1055f7012e2ae5e9090406.jpg (deflated 42%)
  adding: sec_vas/5f111f40cafa33f2dd42eb40.jpg (deflated 22%)
  adding: sec_vas/5f10f943cb51aec63556d31f.jpg (deflated 15%)
  adding: sec_vas/5f11375bbd44bdc9c956e8ce.jpg (deflated 22%)
  adding: sec_vas/5f111cab3064f0e8a20d4cae.jpg (deflated 21%)
  adding:

  adding: sec_vas/5f10e42a11acafc679bf1a0e.jpg (deflated 28%)
  adding: sec_vas/5f104b57d0397ff0dbd3580d.jpg (deflated 53%)
  adding: sec_vas/5f11d4475fdcaa1ca6540927.jpg (deflated 22%)
  adding: sec_vas/5f11c5c2e15bf738604cd9d3.jpg (deflated 15%)
  adding: sec_vas/5f111c8b96792ada6c0b88a5.jpg (deflated 10%)
  adding: sec_vas/5f11d8443be96ab76c2a396c.jpg (deflated 19%)
  adding: sec_vas/5f111f03cafa33f2dd42eb3f.jpg (deflated 8%)
  adding: sec_vas/5f10e27c728b48e7518fe846.jpg (deflated 27%)
  adding: sec_vas/5f10c67e0a6928b1e4faf337.jpg (deflated 17%)
  adding: sec_vas/5f10cbc6832bfc3eafa69f97.jpg (deflated 42%)
  adding: sec_vas/5f10a37e4247d6fe56ac846c.jpg (deflated 31%)
  adding: sec_vas/5f112fbfd48a3b6ae81374fd.jpg (deflated 7%)
  adding: sec_vas/5f11d3092c256c60a6d8b517.jpg (deflated 18%)
  adding: sec_vas/5f11d47b52079cde60987378.jpg (deflated 8%)
  adding: sec_vas/5f11ee87bb5a02d04aed6214.jpg (deflated 20%)
  adding: sec_vas/5f10f66a39a834b3d372ffa7.jpg (deflated 11%)
  adding: s

...

  adding: sec_vas/5f112b422bb1073342ffbefd.jpg (deflated 33%)
  adding: sec_vas/5f11d6f4015cdeecb6355e46.jpg (deflated 23%)
  adding: sec_vas/5f11f38cc551d552820e1e45.jpg (deflated 5%)
  adding: sec_vas/5f10a66b64369eb40e9efd56.jpg (deflated 31%)
  adding: sec_vas/5f104961a9010cccb00a6fb9.jpg (deflated 15%)
  adding: sec_vas/5f1134be647b4c942c1c9430.jpg (deflated 25%)
  adding: sec_vas/5f11d0c9b21163d9e734717a.jpg (deflated 29%)
  adding: sec_vas/5f10496fd2beb944468ac93d.jpg (deflated 12%)
  adding: sec_vas/5f10aa8146593d9f39708954.jpg (deflated 19%)
  adding: sec_vas/5f1126e439a834b3d372ffe6.jpg (deflated 22%)
  adding: sec_vas/5f10f491220d4de6a398065b.jpg (deflated 19%)
  adding: sec_vas/5f10f8d4a8443521dcab27fa.jpg (deflated 38%)
  adding: sec_vas/5f11d38a275ec371c75dca6d.jpg (deflated 32%)
  adding: sec_vas/5f11dd2abdddf55c95bb3726.jpg (deflated 45%)
  adding: sec_vas/5f104d825f996a8ff3988260.jpg (deflated 47%)
  adding: sec_vas/5f1045f25f6293c0cb7cccb3.jpg (deflated 33%)
  adding:

  adding: sec_vas/5f104eef62a91661566b2eef.jpg (deflated 29%)
  adding: sec_vas/5f10ba432b3948fa178430e4.jpg (deflated 6%)
  adding: sec_vas/5f104f081e16a1e46af6efad.jpg (deflated 24%)
  adding: sec_vas/5f1128184230f3edd9233736.jpg (deflated 19%)
  adding: sec_vas/5f104c02693b771c9bc0065d.jpg (deflated 26%)
  adding: sec_vas/5f12286fb9ceca04afa1bada.jpg (deflated 10%)
  adding: sec_vas/5f113113dcd1d6fa1755f895.jpg (deflated 27%)
  adding: sec_vas/5f10cfd3218a8fba944dd2e2.jpg (deflated 20%)
  adding: sec_vas/5f11d33315d0ad8ee3b113ea.jpg (deflated 7%)
  adding: sec_vas/5f10fea0b56b4368d78b379c.jpg (deflated 35%)
  adding: sec_vas/5f10a323e86e1a71cbca46b8.jpg (deflated 35%)
  adding: sec_vas/5f10cb172ce46fbcbe261fd2.jpg (deflated 34%)
  adding: sec_vas/5f11c6ddbde2e720094292f5.jpg (deflated 24%)
  adding: sec_vas/5f11e090cf74ad1c23416907.jpg (deflated 13%)
  adding: sec_vas/5f1140bada8e10b8642638b6.jpg (deflated 14%)
  adding: sec_vas/5f10be686429a4c2a2daa8cd.jpg (deflated 4%)
  adding: s

  adding: sec_vas/5f1122525bf8f5e08e766d2a.jpg (deflated 17%)
  adding: sec_vas/5f11305c578826104845b7df.jpg (deflated 16%)
  adding: sec_vas/5f1114068a5238ede5a6e1cb.jpg (deflated 31%)
  adding: sec_vas/5f10598c0c043804ffc262d5.jpg (deflated 40%)
  adding: sec_vas/5f111bfc6761b3c12a3d741a.jpg (deflated 23%)
  adding: sec_vas/5f11ef2bcc8cf49d20a90ad3.jpg (deflated 19%)
  adding: sec_vas/5f11e7fb777ddd7d5ad0efa6.jpg (deflated 10%)
  adding: sec_vas/5f11d3fe312843ff642a3ab0.jpg (deflated 36%)
  adding: sec_vas/5f10fae0b5600e3ef053353a.jpg (deflated 12%)
  adding: sec_vas/5f12239a7246a8f95926cdfc.jpg (deflated 3%)
  adding: sec_vas/5f12138f5a4a33ef3f20f0f1.jpg (deflated 6%)
  adding: sec_vas/5f111ddf05c6ac9fcdadafda.jpg (deflated 5%)
  adding: sec_vas/5f11ed836ca303aaf43b0c60.jpg (deflated 20%)
  adding: sec_vas/5f11b7673517966342b762ff.jpg (deflated 16%)
  adding: sec_vas/5f118a75ca59186065807d48.jpg (deflated 19%)
  adding: sec_vas/5f1046e0be1cfa76c006a597.jpg (deflated 25%)
  adding: s

  adding: sec_vas/5f10f3bbcafa33f2dd42eaff.jpg (deflated 13%)
  adding: sec_vas/5f10aa8b41393c65d2a695f7.jpg (deflated 30%)
  adding: sec_vas/5f11dfdf055ddedc8492fd3b.jpg (deflated 12%)
  adding: sec_vas/5f104d045744a9178a17f8a7.jpg (deflated 20%)
  adding: sec_vas/5f1110f73d102b9b6c3e23fb.jpg (deflated 28%)
  adding: sec_vas/5f11c0694e8ff82285b70fbd.jpg (deflated 23%)
  adding: sec_vas/5f11d064c1c65949d44de1c4.jpg (deflated 27%)
  adding: sec_vas/5f122faedc420137913c8739.jpg (deflated 11%)
  adding: sec_vas/5f123f3f9a076012fbc4e3a2.jpg (deflated 39%)
  adding: sec_vas/5f10cdd62d4223dcb103eede.jpg (deflated 22%)
  adding: sec_vas/5f10467a508882d89178dee7.jpg (deflated 27%)
  adding: sec_vas/5f10493d12fd9280b68e8884.jpg (deflated 19%)
  adding: sec_vas/5f10cddd7b66b652d1a489e2.jpg (deflated 10%)
  adding: sec_vas/5f10a67877e25c5d2e110df5.jpg (deflated 3%)
  adding: sec_vas/5f1046d396f66f6a7a6b613b.jpg (deflated 19%)
  adding: sec_vas/5f1054485be0a9a25e85a6ec.jpg (deflated 48%)
  adding:

  adding: sec_vas/5f11559485a4c4aeb93b4c7f.jpg (deflated 24%)
  adding: sec_vas/5f108ac289ade70c6d9e2cf8.jpg (deflated 53%)
  adding: sec_vas/5f10f6c5832bfc3eafa69fe0.jpg (deflated 25%)
  adding: sec_vas/5f11375c2c15e22a385c72b6.jpg (deflated 16%)
  adding: sec_vas/5f10a698c744830e33adc6f0.jpg (deflated 9%)
  adding: sec_vas/5f1058cc2d4528c9657cf66b.jpg (deflated 21%)
  adding: sec_vas/5f10cc6c77e25c5d2e110e38.jpg (deflated 12%)
  adding: sec_vas/5f105dbc6f4a0acc390e1388.jpg (deflated 25%)
  adding: sec_vas/5f11ba1748897fc5fa70893d.jpg (deflated 8%)
  adding: sec_vas/5f11e4bff4de88b1efb22013.jpg (deflated 19%)
  adding: sec_vas/5f121e740d9ef156b71d8d63.jpg (deflated 4%)
  adding: sec_vas/5f123d64618b9a804c0e8ddb.jpg (deflated 44%)
  adding: sec_vas/5f11d301ddef473929b26a1a.jpg (deflated 13%)
  adding: sec_vas/5f111b5e05c6ac9fcdadafd0.jpg (deflated 27%)
  adding: sec_vas/5f11adeddc38083580665d23.jpg (deflated 28%)
  adding: sec_vas/5f10586792d471e9d5d8f693.jpg (deflated 13%)
  adding: s

In [92]:
db = db_client["tweets_pipeline_v2"].find({'source': 'def'})


for i in db:
    if "diamondback" in i["entity_prepared"]:
        print(i["_id"], i["entity_prepared"])

5f1130a750aaacd44c4df0d4 ['bp', 'philips', 'exxon', 'shell', 'texas', 'conocophillips', 'devon', 'discovery', 'sm energy', 'qep resources', 'ovintiv', 'williams', 'nyse', 'diamondback', 'mewbourne', 'occidental', 'oneok', 'eog resources', 'apache', 'cimarex', 'ireland', 'texas pacific land trust', 'chevron']
5f1130ac15fecd18b8f382ae ['ovintiv', 'nyse', 'diamondback', 'texas pacific land trust', 'texas']
5f176c24ba9c90aff9347075 ['diamondback']
5f176ca248ae9ad34842b7ca ['index', 'occidental petroleum', 'anadarko', 's&p', 'diamondback', 'oxy', 'etf', 'chevron']
5f176d0abbe44af8adc8e633 ['diamondback']
